<a href="https://colab.research.google.com/github/jhermosillo/Proyecto-IA/blob/main/PROYECTO%20MIB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="font-size:50px;" align="center">Inferencia Bayesiana usando el Motor de Inferencia MIB</h1>

<h1 style="font-size:30px;" align="center">Quien escribió este cuento</h1>

In [66]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import glob
import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize

from collections import Counter  #regresa un diccionario con conteos
from collections import OrderedDict #diccionarios ordenados
from itertools import product   #calcula producto cartesiano

import mib as mb
import tpproc as tp

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Cargar un archivo de excel a colab desde github
!apt-get install subversion
!svn checkout "https://github.com/jhermosillo//Proyecto-IA/trunk/Train/"
!svn checkout "https://github.com/jhermosillo//Proyecto-IA/trunk/Test/"

# Proyecto

In [2]:
archivos = glob.glob('./Train/*/*')
archivos,nombres = tp.carga_cuentos(archivos)
df_train = tp.lee_cuentos(archivos)
df_train.head()

leyendo...
tamaño del contenido de archivos cargados:             1 KB


C:\Users\user\Mi unidad\Clases\IA\BAYESIAN PROGRAMMING\Python_code\Proyecto-IA - GitHub\Proyecto-IA\tpproc.py:84: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace('(','')\
C:\Users\user\Mi unidad\Clases\IA\BAYESIAN PROGRAMMING\Python_code\Proyecto-IA - GitHub\Proyecto-IA\tpproc.py:85: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace(')','')\


,titulo,tipo,autor,texto
0,catástrofes,minicuento,alfonso reyes,pasa hablado cierta catástrofe cósmica amenaza...
1,del perfecto gobernante,minicuento,alfonso reyes,entiende perfecto gobernante perfecto lleno pe...
2,ley profunda,minicuento,alfonso reyes,trata libertarnos simplemente enseñarnos descu...
3,los alivios,minicuento,alfonso reyes,nombres usan castellano restroom impropios bañ...
4,el buen rey león,fábula,esopo,león enojoso cruel violento sino tratable just...


In [3]:
with open("./voc_pry.txt",encoding='utf-8') as f:
    vocabulario = f.read()
vocabulario=vocabulario.split()
print(len(vocabulario))

4763


In [4]:
print(vocabulario[:10])

['vencedor', 'metió', 'apelar', 'gratitud', 'empieza', 'cirujano', 'pellizcó', 'lienzo', 'chorizo', 'mismos']


In [5]:
archivos = glob.glob('./Test/*')
archivos,nombres = tp.carga_cuentos(archivos)
df_test = tp.lee_cuentos(archivos,test=True)
df_test.head()

leyendo...
tamaño del contenido de archivos cargados:             0 KB


C:\Users\user\Mi unidad\Clases\IA\BAYESIAN PROGRAMMING\Python_code\Proyecto-IA - GitHub\Proyecto-IA\tpproc.py:84: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace('(','')\
C:\Users\user\Mi unidad\Clases\IA\BAYESIAN PROGRAMMING\Python_code\Proyecto-IA - GitHub\Proyecto-IA\tpproc.py:85: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace(')','')\


,titulo,texto
0,divina fragilidad,dios creó universo sentía solo eternidad empez...
1,dos seres iguales,cierto día profeta sharía encontró niña jardín...
2,el caballo y el lobo,estación blandos céfiros hacen verdear campos ...
3,el cetro,dijo rey esposa señora verdaderamente reina de...
4,el impostor,cansada esperar hombre llegó puntual sonreírme...


## Data Frames de trabajo

In [6]:
df_train_T = df_train.copy()
df_test_T = df_test.copy()

In [7]:
df_test_T['tokens'] = df_test_T.apply(lambda row: nltk.word_tokenize(row['texto']), axis=1)
df_test_T.head()

,titulo,texto,tokens
0,divina fragilidad,dios creó universo sentía solo eternidad empez...,"[dios, creó, universo, sentía, solo, eternidad..."
1,dos seres iguales,cierto día profeta sharía encontró niña jardín...,"[cierto, día, profeta, sharía, encontró, niña,..."
2,el caballo y el lobo,estación blandos céfiros hacen verdear campos ...,"[estación, blandos, céfiros, hacen, verdear, c..."
3,el cetro,dijo rey esposa señora verdaderamente reina de...,"[dijo, rey, esposa, señora, verdaderamente, re..."
4,el impostor,cansada esperar hombre llegó puntual sonreírme...,"[cansada, esperar, hombre, llegó, puntual, son..."


In [8]:
C=df_test_T.tokens.values
for c in C:
    print(c[:2])

['dios', 'creó']
['cierto', 'día']
['estación', 'blandos']
['dijo', 'rey']
['cansada', 'esperar']
['yendo', 'león']
['ranas', 'decidieron']
['cierto', 'día']
['noche', 'cerrada']
['ibas', 'contar']
['veces', 'vuelven']
['muerte', 'cansada']
['camión', 'detuvo']
['cierto', 'día']
['tres', 'punto']
['narciso', 'hoy']
['cierto', 'día']
['pesar', 'torpor']
['mary', 'dos']


## Preguntas

1. ¿Quién escribió "El caballo y el Lobo"?
2. "La escopeta" ¿es una fábula?
3. ¿Qué probabilidad hay de que "La muerte" haya sido escrita por José Saramago
4. ¿Pudo "La marcha del caracol" haber sido escrita por García Márquez y Fuentes en co-autoría?
5. Dado que "El águila de ala cortada y la zorra" es una fábula, ¿qué probabilidad hay de que haya sido escrita por Jalil Gibrán?
6. ¿Qué proporción hay de minicuentos?
7. ¿Qué cuentos son de Carlos Onetti?
8. ¿Quién dijo: "Desde que abrí los ojos me di cuenta que mi sitio no estaba aquí, donde yo estoy, sino en donde no estoy ni he estado nunca. En alguna parte hay un lugar vacío y ese vacío se llenará de mí y yo me asentaré en ese hueco que insensiblemente rebosará de mí, pleno de mí hasta volverse fuente o surtidor. Y mi vacío, el vacío de mí que soy ahora, se llenará de sí, pleno de sí, pleno de ser hasta los bordes."?
9. ¿Quién es el autor con más cuentos?
10. ¿Qué probablidad hay de quien haya escrito "Retiniblastoma" haya también escrito "El sol y las ranas"?

# Avance de una solución posible

## Datos

In [9]:
df_train_T['Total']=df_train_T.texto.str.split().apply(len)

df_train_T.head()

,titulo,tipo,autor,texto,Total
0,catástrofes,minicuento,alfonso reyes,pasa hablado cierta catástrofe cósmica amenaza...,68
1,del perfecto gobernante,minicuento,alfonso reyes,entiende perfecto gobernante perfecto lleno pe...,120
2,ley profunda,minicuento,alfonso reyes,trata libertarnos simplemente enseñarnos descu...,35
3,los alivios,minicuento,alfonso reyes,nombres usan castellano restroom impropios bañ...,18
4,el buen rey león,fábula,esopo,león enojoso cruel violento sino tratable just...,58


## Reducción del vocabulario

### Vocabulario del conjunto de entrenamiento

#### **Se puede jugar con esta celda para aumentar o reducir aún más el vocabulario**
* La versión anterior tenía muy pocas palabras
* Esta nueva versión aumentó el vocabulario a poco mas de 1,300 palabras
* Sigue siendo bastante menor al tamaño original

In [10]:
s=df_train.texto.str.cat(sep=' ').split()
print(len(s))
conteos=Counter(s)
# print(conteos)
# nuevo_vocabulario = list(set([palabra for palabra in s if conteos[palabra] <= 10 and conteos[palabra]>3 ]))
nuevo_vocabulario = list(set([palabra for palabra in s if conteos[palabra] <= 10 and conteos[palabra]>1 ]))
print(len(nuevo_vocabulario))

7210
1023


In [11]:
df_train_T = df_train.copy()
df_train_T['nuevo_texto']=df_train_T.texto.str.split().\
    apply(lambda texto: [w for w in texto if w in nuevo_vocabulario]).\
    apply(lambda x : ' '.join(x))
df_train_T['nuevo_total']=df_train_T.nuevo_texto.str.split().apply(len)
df_train_T['Conteos']=df_train_T.nuevo_texto.str.split().apply(Counter)


df_train_T.head()

,titulo,tipo,autor,texto,nuevo_texto,nuevo_total,Conteos
0,catástrofes,minicuento,alfonso reyes,pasa hablado cierta catástrofe cósmica amenaza...,pasa cierta catástrofe amenaza fondo catástrof...,26,"{'pasa': 1, 'cierta': 1, 'catástrofe': 3, 'ame..."
1,del perfecto gobernante,minicuento,alfonso reyes,entiende perfecto gobernante perfecto lleno pe...,entiende perfecto gobernante perfecto lleno pe...,56,"{'entiende': 1, 'perfecto': 3, 'gobernante': 4..."
2,ley profunda,minicuento,alfonso reyes,trata libertarnos simplemente enseñarnos descu...,debe ley lleva pasa saber peso alma suma conoc...,10,"{'debe': 1, 'ley': 1, 'lleva': 1, 'pasa': 1, '..."
3,los alivios,minicuento,alfonso reyes,nombres usan castellano restroom impropios bañ...,nombres usan baño nombre alivios ido vuelve al...,8,"{'nombres': 1, 'usan': 1, 'baño': 1, 'nombre':..."
4,el buen rey león,fábula,esopo,león enojoso cruel violento sino tratable just...,león cruel violento sino justo buena llegó rey...,26,"{'león': 1, 'cruel': 1, 'violento': 1, 'sino':..."


In [12]:
print(Counter(df_train_T.iloc[0].texto.split()))
print(Counter(df_train_T.iloc[0].nuevo_texto.split()))
print(df_train_T.iloc[0].Conteos)

Counter({'catástrofe': 3, 'parte': 2, 'pasa': 1, 'hablado': 1, 'cierta': 1, 'cósmica': 1, 'amenaza': 1, 'fondo': 1, 'universo': 1, 'ancorajes': 1, 'fragmento': 1, 'pues': 1, 'sucede': 1, 'alguna': 1, 'distante': 1, 'ignorada': 1, 'repercute': 1, 'cofrecito': 1, 'vibratorio': 1, 'corazón': 1, 'muchas': 1, 'veces': 1, 'sabemos': 1, 'rara': 1, 'inquietud': 1, 'traviesa': 1, 'dolor': 1, 'inesperado': 1, 'malestar': 1, 'llega': 1, 'ser': 1, 'físico': 1, 'corpóreo': 1, 'mejor': 1, 'reventado': 1, 'cometa': 1, 'estallado': 1, 'nebulosa': 1, 'viento': 1, 'energías': 1, 'etéreas': 1, 'desatado': 1, 'varios': 1, 'millones': 1, 'añosluz': 1, 'ahora': 1, 'llegando': 1, 'casa': 1, 'nubes': 1, 'radiantes': 1, 'visitan': 1, 'puede': 1, 'extinguirse': 1, 'estrella': 1, 'paguemos': 1, 'aún': 1, 'merecerlo': 1, 'vivimos': 1, 'morimos': 1, 'asaetados': 1, 'oscuras': 1, 'flechas': 1, 'arqueros': 1, 'sombras': 1, 'sitiados': 1})
Counter({'catástrofe': 3, 'parte': 2, 'pasa': 1, 'cierta': 1, 'amenaza': 1, 'f

In [13]:
print(df_train_T.iloc[3].texto)
print(df_train_T.iloc[3].nuevo_texto)

nombres usan castellano restroom impropios baño abominables proponemos nombre inocuo evocador alivios adónde ido fulana ahora vuelve alivios
nombres usan baño nombre alivios ido vuelve alivios


#### Tamaño del vocabulario de entrenamiento

In [14]:
print(len(nuevo_vocabulario))

1023


### Vocabulario del conjunto de prueba

In [15]:
s=df_test.texto.str.cat(sep=' ').split()
print(len(s))
conteos=Counter(s)
# print(conteos)
nuevo_vocab = list(set([palabra for palabra in s if conteos[palabra] <= 10 and conteos[palabra]>1 ]))
print(len(nuevo_vocab))

2224
336


In [16]:
df_test_T = df_test.copy()
df_test_T['nuevo_texto']=df_test_T.texto.str.split().\
    apply(lambda texto: [w for w in texto if w in nuevo_vocab]).\
    apply(lambda x : ' '.join(x))
df_test_T['nuevo_total']=df_test_T.nuevo_texto.str.split().apply(len)
df_test_T['Conteos']=df_test_T.nuevo_texto.str.split().apply(Counter)


df_test_T.head()

,titulo,texto,nuevo_texto,nuevo_total,Conteos
0,divina fragilidad,dios creó universo sentía solo eternidad empez...,dios universo sentía empezó sentía cosa tan un...,90,"{'dios': 9, 'universo': 3, 'sentía': 2, 'empez..."
1,dos seres iguales,cierto día profeta sharía encontró niña jardín...,cierto profeta sharía encontró niña jardín niñ...,92,"{'cierto': 1, 'profeta': 7, 'sharía': 4, 'enco..."
2,el caballo y el lobo,estación blandos céfiros hacen verdear campos ...,campos vida cierto lobo alegría buena seguida ...,34,"{'campos': 1, 'vida': 1, 'cierto': 1, 'lobo': ..."
3,el cetro,dijo rey esposa señora verdaderamente reina de...,rey esposa señora reina ser esposa señor rey s...,35,"{'rey': 3, 'esposa': 2, 'señora': 1, 'reina': ..."
4,el impostor,cansada esperar hombre llegó puntual sonreírme...,cansada esperar hombre llegó primer nombre rep...,83,"{'cansada': 1, 'esperar': 1, 'hombre': 2, 'lle..."


#### Tamaño del vocabulario de prueba

In [17]:
print(len(nuevo_vocab))
print(df_test_T.iloc[0].texto)
print(df_test_T.iloc[0].nuevo_texto)

336
dios creó universo sentía solo eternidad empezó solo sentía solo necesitaba inventar cosa tan complicada universo dios contado incluso espectáculo magnífico nebulosas agujeros negros tal sentimiento soledad persistiese atormentarlo pensó pensó cabo pensar hizo mujer imagen semejanza después habiéndola hecho vio bueno tarde comprendió solo curaría definitivamente mal solo acostándose verificó aún mejor pasado algún tiempo posible saber previsión accidente biológico mente divina nació niño imagen semejanza dios niño creció convirtió joven hombre ahora bien dios pasó cabeza simple idea crear mujer dar joven sentimiento soledad afligido padre tardó repetirse hijo ahí entró diablo esperar primer impulso dios acabar ahí mismo incestuosa especie entró repente cansancio fastidio tener repetir creación hecho universo parecía tan magnífico dirá siendo dios podía hacer cuantos universos quisiese equivale desconocer naturaleza profunda dios lógicamente hecho mejor universos posibles podía hace

#### Tamaño del vocabulario conjunto (total de palabras)

In [39]:
vocabulario = list(set(nuevo_vocabulario+nuevo_vocab))
print(len(vocabulario))

1207


## Diccionario del vocabulario

In [47]:
voc_dict = OrderedDict(zip(vocabulario,range(len(vocabulario))))
v = list(voc_dict.keys())
i=0
while i<10:
    print(v[i], end=', ')
    i+=1

fieras, rebuscador, estrella, jabón, bien, volví, decir, agente, repitió, dije, 

# Modelo

## Suponiendo un modelo de descomposición exacta:
### $P(A)P(T|A)\displaystyle \prod P(W|AT)$

## Variables

### Conteo de autores y tipos

In [41]:
# Ocurrencias de cada autor y conteos de ocurrencias y de número de autores
oc_autor = Counter(df_train_T.autor)
tot_oc_aut = np.sum(list(oc_autor.values()))
tot_autores = len(oc_autor)

print(oc_autor)
print(tot_oc_aut)
print(tot_autores)

# Ocurrencias de cada tipo y conteos de ocurrencias y de número de tipos
oc_tipo = Counter(df_train_T.tipo)
tot_oc_tipo = np.sum(list(oc_tipo.values()))
tot_tipos = len(oc_tipo)

print(oc_tipo)
print(tot_oc_tipo)  #debe ser igual a tot_oc_aut
print(tot_tipos)

Counter({'gibrán jalil gibrán': 5, 'alfonso reyes': 4, 'gabriel garcía márquez': 4, 'jean de la fontaine': 4, 'esopo': 3, 'carlos fuentes': 2, 'juan carlos onetti': 2, 'horacio quiroga': 2, 'juan rulfo': 2, 'josé saramago': 2})
30
10
Counter({'minicuento': 14, 'cuento': 13, 'fábula': 3})
30
3


### Definición de variables MIB

#### **Nota Importante**
* Por ahora, el motor de inferencia sólo acepta valores numéricos como valores posibles de una variable.
* Por lo tanto, vamos a crear un mapeo entre los nombres de los autores y un valor entero mediante un diccionario.
* También creamos un mapeo inverso, para poder recuperar las etiquetas a partir de los enteros.
* Haremos lo mismo con los tipos.

In [42]:
#Autor
autor_val = dict(zip(oc_autor.keys(),range(len(oc_autor.keys()))))
val_autor = dict(zip(range(len(oc_autor.keys())),oc_autor.keys()))
print(autor_val)
print(val_autor)
A=mb.Var('A')
A.set_values(range(len(autor_val)))
print(A.print_var())

#Tipo
tipo_val = dict(zip(oc_tipo.keys(),range(len(oc_tipo.keys()))))
val_tipo = dict(zip(range(len(oc_tipo.keys())),oc_tipo.keys()))
T=mb.Var('T')
T.set_values(range(len(oc_tipo.keys())))
print(T.print_var())

#Palabras, presente:1 ausente:0
#Se crean de manera similar a como se hizo para el filtro SPAM
W = {}
for w in voc_dict:    #vocabulario reducido
    i = voc_dict[w]
    W[i] = {w:mb.Var(name='W'+str(i),values=[0,1])}
print(len(W))
for i in range(10):
    print(f'W[{i}]: {W[i]}')

{'alfonso reyes': 0, 'esopo': 1, 'carlos fuentes': 2, 'gabriel garcía márquez': 3, 'gibrán jalil gibrán': 4, 'jean de la fontaine': 5, 'juan carlos onetti': 6, 'horacio quiroga': 7, 'juan rulfo': 8, 'josé saramago': 9}
{0: 'alfonso reyes', 1: 'esopo', 2: 'carlos fuentes', 3: 'gabriel garcía márquez', 4: 'gibrán jalil gibrán', 5: 'jean de la fontaine', 6: 'juan carlos onetti', 7: 'horacio quiroga', 8: 'juan rulfo', 9: 'josé saramago'}
('A', 10, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
('T', 3, [0, 1, 2])
1207
W[0]: {'fieras': <mib.Var object at 0x0000019356CB5F48>}
W[1]: {'rebuscador': <mib.Var object at 0x0000019356CB5DC8>}
W[2]: {'estrella': <mib.Var object at 0x0000019356CB5A48>}
W[3]: {'jabón': <mib.Var object at 0x0000019356CB5A88>}
W[4]: {'bien': <mib.Var object at 0x0000019356CB5A08>}
W[5]: {'volví': <mib.Var object at 0x00000193556C3488>}
W[6]: {'decir': <mib.Var object at 0x00000193559F8308>}
W[7]: {'agente': <mib.Var object at 0x00000193559F8EC8>}
W[8]: {'repitió': <mib.Var object at 0

## Identificación de parámetros (estadísticas)

### Método:
1. Descomposición exacta: $P(A)P(T|A)P(W|AT)$
2. Calcular las estadísticas por cada distribución de la descomposición
3. Para la variable Autor $P(A)$:
    - 3.1 Contar el número total de ocurrencias de autores en Train: $n_a$.
    - 3.2 Contar el número de ocurrencias de cada autor en Train: $n^i$.
    - 3.3 Dividir este número entre el número total de ocurrencias de autores en Train: $\displaystyle \frac{n^i}{n_a}$.
4. Para la variable Tipo $P(T|A)$:
    - 4.1 Obtener el conjunto de autores (valores únicos) $a$; entradas a la Tabla.
    - 4.2 Obtener el conjunto de tipos (valores únicos) $j$; estas son las salidas de la Tabla.
    - 4.3 Calcular el producto cartesiano de las combinaciones posibles (a,j).
    - 4.4 Contar el número de ocurrencias totales de cada $j$ por cada autor: $n^j_a$.
    - 4.5 Dividir este número entre el número total de tipos por autor: $\displaystyle \frac{n^j_a}{\sum n^j_a}$.
    - 4.6 Aplicar la corrección de Laplace para evitar probabilidades en $0$.
5. Para las variables W (palabras) $P(W|AT)$: **NOTA** se trata de calcular N Tablas, donde N es el tamaño del vocabulario.
    - 5.1 Calcular el producto cartesiano de las combinaciones posibles (a,j); estas son las entradas a cada Tabla.
    - 5.2 Por cada combinación, calcular la probabilidad de cada palabra del vocabulario, en función de su ocurrencia en esa combinación, aplicando la corrección de Laplace. 

## $P(A)$
   - Contar el número total de ocurrencias de autores en Train: $n_a$: _tot_oc_aut_
   - Contar el número de ocurrencias de cada autor en Train: $n^i$: _oc_autor_ 
   - Dividir este número entre el número total de ocurrencias de autores en Train: $\displaystyle \frac{n^i}{n_a}$

In [21]:
n_=[]
for autor in oc_autor:
    n_.append((autor,oc_autor[autor]/tot_oc_aut))

#Dicionario de valores de probabilidad
dA = dict(n_)
#Distribución de probabilidad
PA=mb.Distrib(name='P(A)',variable=[A],tabla=dA)
PA._print()

Printing:  P(A)
('A', 0):0.13333333333333333  ('A', 1):0.1  ('A', 2):0.06666666666666667  ('A', 3):0.13333333333333333  ('A', 4):0.16666666666666666  ('A', 5):0.13333333333333333  ('A', 6):0.06666666666666667  ('A', 7):0.06666666666666667  ('A', 8):0.06666666666666667  ('A', 9):0.06666666666666667  



## $P(T|A)$
   - Obtener el conjunto de autores (valores únicos) $a$; entradas a la Tabla: _df_train.autor_
   - Obtener el conjunto de tipos (valores únicos) $j$; estas son las salidas de la Tabla: _df_train.tipo_ 
   - Calcular el producto cartesiano de las combinaciones posibles (a,j): _pares_ta_
   - Contar el número de ocurrencias totales de cada $j$ por cada autor: $n^j_a$: _conteo_pares_
   - Dividir este número entre el número total de tipos por autor: $\displaystyle \frac{n^j_a}{\sum n^j_a}$.
   - Aplicar la corrección de Laplace para evitar probabilidades en $0$.

In [22]:
# Ocurrencias (conteos) de cada combinación (tipo,autor)
conteo_pares = Counter(zip(df_train_T.autor,df_train_T.tipo))
# print(conteo_pares)

# Combinaciones (tipo,autor)
autores = list(set(df_train_T.autor))
tipos = list(set(df_train_T.tipo))
pares_ta = product(autores,tipos)

# Llenado preliminar de la Tabla de probabilidades con conteos por tipo dado el autor
# Se usan claves numéricas definidas más arriba: {'alfonso reyes': 0, 'esopo': 1, 
#'carlos fuentes': 2, 'gabriel garcía márquez': 3, 'gibrán jalil gibrán': 4, 
#'jean de la fontaine': 5, 'juan carlos onetti': 6, 'horacio quiroga': 7, 
#'juan rulfo': 8, 'josé saramago': 9}
dT_A = {}
for autor, tipo in pares_ta:
    par = (autor,tipo)
    av = autor_val[autor]
    if av in dT_A.keys():
        if par in conteo_pares.keys():
            dT_A[av][tipo_val[tipo]]=conteo_pares[par]
        else:
            dT_A[av][tipo_val[tipo]]=0            
    else:
        if par in conteo_pares.keys():
            dT_A[av]={tipo_val[tipo]:conteo_pares[par]}
        else:
            dT_A[av]={tipo_val[tipo]:0}
# print(dT_A)

# Corrección de Laplace en el caso general
# Se corrige la misma tabla que se senvía como argumento; no se crea una nueva.
def Laplace_gral(tabla):
    n = len(tabla[list(tabla.keys())[0]]) 
    for k in tabla.keys():
        registro = tabla[k]
        n_j = np.sum(list(registro.values()))
        for i in registro.keys():
            pb = (registro[i] + 1)/(n_j+n)
            tabla[k][i]=pb
    return

Laplace_gral(dT_A)
# print(dT_A)

#Distribución de probabilidad
PT_A=mb.DistribCond('P(T|A)',[T],[A],dT_A)
PT_A._print()

Printing:  P(T|A)
('A', 0):{('T', 0): 0.25, ('T', 1): 0.125, ('T', 2): 0.625}  ('A', 1):{('T', 0): 0.7142857142857143, ('T', 1): 0.14285714285714285, ('T', 2): 0.14285714285714285}  ('A', 2):{('T', 0): 0.16666666666666666, ('T', 1): 0.6666666666666666, ('T', 2): 0.16666666666666666}  ('A', 3):{('T', 0): 0.2, ('T', 1): 0.2, ('T', 2): 0.6}  ('A', 4):{('T', 0): 0.2, ('T', 1): 0.2, ('T', 2): 0.6}  ('A', 5):{('T', 0): 0.4, ('T', 1): 0.2, ('T', 2): 0.4}  ('A', 6):{('T', 0): 0.5714285714285714, ('T', 1): 0.14285714285714285, ('T', 2): 0.2857142857142857}  ('A', 7):{('T', 0): 0.4, ('T', 1): 0.2, ('T', 2): 0.4}  ('A', 8):{('T', 0): 0.7142857142857143, ('T', 1): 0.14285714285714285, ('T', 2): 0.14285714285714285}  ('A', 9):{('T', 0): 0.2, ('T', 1): 0.2, ('T', 2): 0.6}  



## $P(W|AT)$
   - Calcular el producto cartesiano de las combinaciones posibles (a,j); estas son las entradas a cada Tabla.
   - Por cada combinación, calcular la probabilidad de cada palabra del vocabulario, en función de su ocurrencia en esa combinación, aplicando la corrección de Laplace. 

#### Procedemos de manera similar a como lo hicimos para el modelo de filtrado SPAM.
* Las palabras a tomar en cuenta son las del vocabulario.
* Se deben codificar las combinaciones (a,t) en valores numéricos.
* Por cada combinación (a,t):
    - si existe una historia (nuevo_texto), entonces cada palabra de este cuento está presente, y el resto de las palabras del vocabulario están ausentes.
    - si no hay historia, entonces ninguna palabra del vocabulario está presente --> Probabilidad 0 para todo el vocabulario.
* Al aplicar la corrección de Laplace, evitamos que una combinación (a,t), que por ahora no tiene texto, se considere imposible, al asignarle una probabilidad mínima a cada palabra del vocabulario.
* Para facilitar el "conteo" (presencia/ausencia) haremos un diccionario del vocabulario, incialmente en ceros.

In [63]:
# Combinaciones (tipo,autor)
autores = list(set(df_train_T.autor))
tipos = list(set(df_train_T.tipo))
pares_ta = list(product(autores,tipos))

# Codificación de los pares (a,t) usando los valores numéricos pre-asignados
def encode_paresTA(pares_ta):
    paresTA_encoded = {}
    for par in pares_ta:
        autor,tipo = par
        acode = autor_val[autor]
        tcode = tipo_val[tipo]
        paresTA_encoded[par] = (acode,tcode)
    return paresTA_encoded

#Conteos de palabras en nuevo_texto por pares (a,t) en el data frame de train
conteo_w = dict(df_train_T.Conteos)
pares_train = list(zip(df_train_T.autor,df_train_T.tipo))

for i,k in enumerate(conteo_w.keys()):
    conteo_w[k] = {pares_train[i] : dict(conteo_w[k])}

# Probabilidades (presencia/ausencia) de palabras en cada combinación (a,t)
# usando corrección de Laplace
def check(Word,pares_ta):
    pw_ = {}
    palabra = list(Word.keys())[0]
    var = Word[palabra]
    # print(var)
    # print(palabra)
    pares_ta = encode_paresTA(pares_ta)
    for par in pares_ta:
        cw_1 = 0   # conteos de presencia 
        cw_0 = 0   # y ausencia en 0
        it = 0     # total de pares (a,t) contabilizados
        # print('\n',par)
        for item in list(conteo_w.values()):  # checamos cada par (a,t) en el conjunto de train
            # print(i,list(item.keys())[0], end=' ')
            it += 1
            if par in item: # checamos si el par (a,t) de entrada tiene correspondencia en Train
                if palabra in list(item.values())[0]: #checamos si la palabra existe en el par (a,t) de Train
                    cw_1 += 1
                else:
                    cw_0 += 1  #debemos contar también la no existencia por si hay igualdad
        if cw_0 == cw_1: # la palabra aparece por igual en todos los pares Train, o bien, no hay pares Train
            pw_1 = 0.5   #corrección de Laplace que indica que la palabra puede o no estar por igual
            pw_0 = 0.5
        else:
            pw_1=(1+cw_1)/(var.card+it) #corrección de Laplace en caso de que la palabra exista
            pw_0 = 1-pw_1
        pw_[pares_ta[par]]=[pw_0,pw_1]
    pw_ = OrderedDict(sorted(pw_.items()))
    return pw_

# Descomentar para probar y comentar el siguiente bloque
# pares_ta = encode_paresTA(pares_ta)
# t = check(W[0],pares_ta)
# print(len(t),t)

pares_ta = encode_paresTA(pares_ta)
PW_AT=[]
for w in voc_dict:    #vocabulario reducido
    i = voc_dict[w]
    t = check(W[i],pares_ta)
    PW_AT.append(mb.DistribCond(name='P(W'+str(i)+'|AT)',var=W[i][vocabulario[i]],indep=[A,T],tabla=t))

In [64]:
print(vocabulario[0])
print(PW_AT[0].name,'\n',PW_AT[0].to_Frame(),'\n')

fieras
P(W0|AT) 
               (W0, 0)  (W0, 1)
(A, T, 0, 0)  0.96875  0.03125
(A, T, 0, 1)  0.50000  0.50000
(A, T, 0, 2)  0.50000  0.50000
(A, T, 1, 0)  0.50000  0.50000
(A, T, 1, 1)  0.96875  0.03125
(A, T, 1, 2)  0.50000  0.50000
(A, T, 2, 0)  0.96875  0.03125
(A, T, 2, 1)  0.50000  0.50000
(A, T, 2, 2)  0.96875  0.03125
(A, T, 3, 0)  0.96875  0.03125
(A, T, 3, 1)  0.50000  0.50000
(A, T, 3, 2)  0.96875  0.03125
(A, T, 4, 0)  0.96875  0.03125
(A, T, 4, 1)  0.50000  0.50000
(A, T, 4, 2)  0.96875  0.03125
(A, T, 5, 0)  0.96875  0.03125
(A, T, 5, 1)  0.50000  0.50000
(A, T, 5, 2)  0.50000  0.50000
(A, T, 6, 0)  0.50000  0.50000
(A, T, 6, 1)  0.50000  0.50000
(A, T, 6, 2)  0.96875  0.03125
(A, T, 7, 0)  0.50000  0.50000
(A, T, 7, 1)  0.50000  0.50000
(A, T, 7, 2)  0.96875  0.03125
(A, T, 8, 0)  0.50000  0.50000
(A, T, 8, 1)  0.50000  0.50000
(A, T, 8, 2)  0.96875  0.03125
(A, T, 9, 0)  0.96875  0.03125
(A, T, 9, 1)  0.50000  0.50000
(A, T, 9, 2)  0.96875  0.03125 



In [65]:
print(vocabulario[1])
print(PW_AT[1].name,'\n',PW_AT[1].to_Frame(),'\n')

rebuscador
P(W1|AT) 
               (W1, 0)  (W1, 1)
(A, T, 0, 0)  0.96875  0.03125
(A, T, 0, 1)  0.50000  0.50000
(A, T, 0, 2)  0.50000  0.50000
(A, T, 1, 0)  0.50000  0.50000
(A, T, 1, 1)  0.96875  0.03125
(A, T, 1, 2)  0.50000  0.50000
(A, T, 2, 0)  0.96875  0.03125
(A, T, 2, 1)  0.50000  0.50000
(A, T, 2, 2)  0.96875  0.03125
(A, T, 3, 0)  0.96875  0.03125
(A, T, 3, 1)  0.50000  0.50000
(A, T, 3, 2)  0.96875  0.03125
(A, T, 4, 0)  0.96875  0.03125
(A, T, 4, 1)  0.50000  0.50000
(A, T, 4, 2)  0.96875  0.03125
(A, T, 5, 0)  0.96875  0.03125
(A, T, 5, 1)  0.50000  0.50000
(A, T, 5, 2)  0.50000  0.50000
(A, T, 6, 0)  0.50000  0.50000
(A, T, 6, 1)  0.50000  0.50000
(A, T, 6, 2)  0.96875  0.03125
(A, T, 7, 0)  0.50000  0.50000
(A, T, 7, 1)  0.50000  0.50000
(A, T, 7, 2)  0.50000  0.50000
(A, T, 8, 0)  0.50000  0.50000
(A, T, 8, 1)  0.50000  0.50000
(A, T, 8, 2)  0.96875  0.03125
(A, T, 9, 0)  0.96875  0.03125
(A, T, 9, 1)  0.50000  0.50000
(A, T, 9, 2)  0.96875  0.03125 



## Preguntas

1. ¿Quién escribió "El caballo y el Lobo"?
2. "La escopeta" ¿es una fábula?
3. ¿Qué probabilidad hay de que "La muerte" haya sido escrita por José Saramago
4. ¿Pudo "La marcha del caracol" haber sido escrita por García Márquez y Fuentes en co-autoría?
5. Dado que "El águila de ala cortada y la zorra" es una fábula, ¿qué probabilidad hay de que haya sido escrita por Jalil Gibrán?
6. ¿Qué proporción hay de minicuentos?
7. ¿Qué cuentos son de Carlos Onetti?
8. ¿Quién dijo: "Desde que abrí los ojos me di cuenta que mi sitio no estaba aquí, donde yo estoy, sino en donde no estoy ni he estado nunca. En alguna parte hay un lugar vacío y ese vacío se llenará de mí y yo me asentaré en ese hueco que insensiblemente rebosará de mí, pleno de mí hasta volverse fuente o surtidor. Y mi vacío, el vacío de mí que soy ahora, se llenará de sí, pleno de sí, pleno de ser hasta los bordes."?
9. ¿Quién es el autor con más cuentos?
10. ¿Qué probablidad hay de quien haya escrito "Retiniblastoma" haya también escrito "El sol y las ranas"?

In [27]:
df_test_T.head()

,titulo,texto,nuevo_texto,nuevo_total,Conteos
0,divina fragilidad,dios creó universo sentía solo eternidad empez...,dios universo sentía empezó sentía cosa tan un...,90,"{'dios': 9, 'universo': 3, 'sentía': 2, 'empez..."
1,dos seres iguales,cierto día profeta sharía encontró niña jardín...,cierto profeta sharía encontró niña jardín niñ...,92,"{'cierto': 1, 'profeta': 7, 'sharía': 4, 'enco..."
2,el caballo y el lobo,estación blandos céfiros hacen verdear campos ...,campos vida cierto lobo alegría buena seguida ...,34,"{'campos': 1, 'vida': 1, 'cierto': 1, 'lobo': ..."
3,el cetro,dijo rey esposa señora verdaderamente reina de...,rey esposa señora reina ser esposa señor rey s...,35,"{'rey': 3, 'esposa': 2, 'señora': 1, 'reina': ..."
4,el impostor,cansada esperar hombre llegó puntual sonreírme...,cansada esperar hombre llegó primer nombre rep...,83,"{'cansada': 1, 'esperar': 1, 'hombre': 2, 'lle..."
